In [4]:
import numpy as np
import pandas as pd
import os

# 1. Problem:

“How can I optimize my listing to attract more bookings and maximize revenue?”

# Causes for Analysis:

Understand pricing trends to remain competitive.

Identify high-demand neighborhoods.

See how availability and minimum nights affect reviews and bookings.

Learn from top-performing listings (e.g., review patterns, room types).

# 2. Problem:

Which listings are violating platform rules, underperforming, or affecting customer satisfaction?”

# Causes for Analysis:

Detect unlicensed listings or non-compliant hosts.

Monitor market saturation by location or host.

Identify potentially fraudulent listings (e.g., 0 reviews, high prices, long minimum stays).



In [5]:

url = r"C:\Users\USER\Desktop\Cube\Data_Science\Project\listings"
os.chdir(url)

listing = pd.read_csv("listings.csv")
listing.head(30)

FileNotFoundError: [Errno 2] No such file or directory: 'listings.csv'

In [ ]:
listing.shape

(37434, 18)

In [ ]:
listing.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

In [ ]:
listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37434 entries, 0 to 37433
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37434 non-null  int64  
 1   name                            37432 non-null  object 
 2   host_id                         37434 non-null  int64  
 3   host_name                       37396 non-null  object 
 4   neighbourhood_group             37434 non-null  object 
 5   neighbourhood                   37434 non-null  object 
 6   latitude                        37434 non-null  float64
 7   longitude                       37434 non-null  float64
 8   room_type                       37434 non-null  object 
 9   price                           22308 non-null  float64
 10  minimum_nights                  37434 non-null  int64  
 11  number_of_reviews               37434 non-null  int64  
 12  last_review                     

In [ ]:
listing.isnull().sum()

id                                    0
name                                  2
host_id                               0
host_name                            38
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                             15126
minimum_nights                        0
number_of_reviews                     0
last_review                       11787
reviews_per_month                 11787
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                           32065
dtype: int64

# Data Cleaning

In [ ]:
listing.head().T

,0,1,2,3,4
id,2595,6848,6872,6990,7064
name,Skylit Midtown Castle Sanctuary,Only 2 stops to Manhattan studio,Uptown Sanctuary w/ Private Bath (Month to Month),UES Beautiful Blue Room,"Amazing location! Wburg. Large, bright & tranquil"
host_id,2845,15991,16104,16800,17297
host_name,Jennifer,Allen & Irina,Kahshanna,Cyn,Joelle
neighbourhood_group,Manhattan,Brooklyn,Manhattan,Manhattan,Brooklyn
neighbourhood,Midtown,Williamsburg,East Harlem,East Harlem,Williamsburg
latitude,40.75356,40.70935,40.80107,40.78778,40.71248
longitude,-73.98559,-73.95342,-73.94255,-73.94759,-73.95881
room_type,Entire home/apt,Entire home/apt,Private room,Private room,Private room
price,NaN,96.0,59.0,67.0,NaN


In [ ]:
# Renaming Columns
listing.rename(
    columns={
        "number_of_reviews": "total_reviews",
        "calculated_host_listings_count": "host_listings_count",
    },
    inplace=True
)

In [ ]:
# Checking for duplicates
listing.duplicated().sum()

NameError: name 'listing' is not defined

In [ ]:
# Handling missing values
missing_values_perc = listing.isnull().mean() * 100
missing_values_perc[missing_values_perc > 0].sort_values(ascending=False)

license              85.657424
price                40.407117
last_review          31.487418
reviews_per_month    31.487418
host_name             0.101512
name                  0.005343
dtype: float64

In [ ]:
# Imputing missing values
# For 'host_name', we can fill with 'Unknown' & 'name' with 'no_name'

import warnings
warnings.filterwarnings("ignore")

listing['host_name'].fillna('Unknown', inplace=True)
listing['name'].fillna('no_name', inplace=True)

In [ ]:
# For 'reviews_per_month', we can fill with '0'
listing['reviews_per_month'].fillna(0, inplace=True)

In [ ]:
# Dropping missing values in the target column 'price'
listing.dropna(subset=['price'], inplace=True)


In [ ]:
listing.shape

(22308, 18)

In [ ]:
# Creating a new column 'has_license'
listing['has_license'] = listing['license'].apply(lambda x: 1 if pd.notnull(x) else 0)

# dropping the 'license' column as it is no longer needed
listing.drop(columns=['license'], inplace=True)

NameError: name 'listing' is not defined

In [ ]:
# Creating a binary indicators for last_review
listing['last_review'] = pd.to_datetime(listing['last_review'], errors='coerce')
listing['last_review_num'] = listing['last_review'].apply(
    lambda x: 1 if pd.notnull(x) else 0
)   
# Dropping the 'last_review' column as it is no longer needed
listing.drop(columns=['last_review'], inplace=True)

NameError: name 'pd' is not defined

In [ ]:
# Replace null values in 'last_review' with 'No Review'

listing['last_review_num'].fillna('No Review', inplace=True)


In [ ]:
listing.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,total_reviews,reviews_per_month,host_listings_count,availability_365,number_of_reviews_ltm,has_license,last_review_num
1,6848,Only 2 stops to Manhattan studio,15991,Allen & Irina,Brooklyn,Williamsburg,40.709350,-73.953420,Entire home/apt,96.0,30,194,1.01,1,173,3,0,1
2,6872,Uptown Sanctuary w/ Private Bath (Month to Month),16104,Kahshanna,Manhattan,East Harlem,40.801070,-73.942550,Private room,59.0,30,1,0.03,2,83,0,0,1
3,6990,UES Beautiful Blue Room,16800,Cyn,Manhattan,East Harlem,40.787780,-73.947590,Private room,67.0,30,251,1.34,1,264,5,0,1
5,7097,"Perfect for Your Parents, With Garden & Patio",17571,Jane,Brooklyn,Fort Greene,40.691940,-73.973890,Private room,195.0,2,398,2.16,2,190,36,1,1
6,7801,Sunny Williamsburg Loft with Sauna,21207,Chaya,Brooklyn,Williamsburg,40.718807,-73.956177,Entire home/apt,290.0,30,13,0.07,1,0,1,0,1


In [ ]:
# checking for categories in columns
listing['last_review_num'].value_counts()

last_review_num
1    15510
0     6798
Name: count, dtype: int64

In [ ]:
# dropping unnecessary columns
listing.drop(columns=['id', 'host_id', 'last_review_num'], inplace=True)

In [ ]:
listing['room_type'].value_counts()

room_type
Entire home/apt    12664
Private room        9186
Hotel room           372
Shared room           86
Name: count, dtype: int64

In [ ]:
listing['neighbourhood_group'].value_counts()

neighbourhood_group
Manhattan        10205
Brooklyn          7455
Queens            3420
Bronx              912
Staten Island      316
Name: count, dtype: int64

In [ ]:
categorical_columns = [
    'neighbourhood_group',
    'neighbourhood',
    'room_type']
# Converting categorical columns to 'category' dtype
for col in categorical_columns:
    listing[col] = listing[col].astype('category')


# Exploratory Data Analysis (EDA)

In [ ]:
listing.head()

,name,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,total_reviews,reviews_per_month,host_listings_count,availability_365,number_of_reviews_ltm,has_license
1,Only 2 stops to Manhattan studio,Allen & Irina,Brooklyn,Williamsburg,40.709350,-73.953420,Entire home/apt,96.0,30,194,1.01,1,173,3,0
2,Uptown Sanctuary w/ Private Bath (Month to Month),Kahshanna,Manhattan,East Harlem,40.801070,-73.942550,Private room,59.0,30,1,0.03,2,83,0,0
3,UES Beautiful Blue Room,Cyn,Manhattan,East Harlem,40.787780,-73.947590,Private room,67.0,30,251,1.34,1,264,5,0
5,"Perfect for Your Parents, With Garden & Patio",Jane,Brooklyn,Fort Greene,40.691940,-73.973890,Private room,195.0,2,398,2.16,2,190,36,1
6,Sunny Williamsburg Loft with Sauna,Chaya,Brooklyn,Williamsburg,40.718807,-73.956177,Entire home/apt,290.0,30,13,0.07,1,0,1,0


In [ ]:

import plotly.express as px
df_grouped = listing.groupby('neighbourhood_group', as_index=False)['price'].mean()

fig = px.bar(
    df_grouped,
    x='neighbourhood_group',
    y='price',
    color='neighbourhood_group',
    title='Average Price by Neighbourhood Group',
    labels={'price': 'Average Price ($)', 'neighbourhood_group': 'Neighbourhood Group'}
)
fig.update_layout(
    xaxis_title='Neighbourhood Group',
    yaxis_title='Average Price ($)',
    template='plotly_white'
)
fig.show()


From the plot we can conclude that Manhattan is the most expensive neighboorhood and Bronx the least expensive among them

In [ ]:
# Distribution by 'room_type'
df_room_type = listing.groupby('room_type', as_index=False)['price'].mean()

fig = px.bar(
    df_room_type,
    x='room_type',
    y='price',
    color='room_type',
    title='Average Price by Room Type',
    labels={'price': 'Average Price ($)', 'room_type': 'Room Type'}
)
fig.update_layout(
    xaxis_title='Room Type',
    yaxis_title='Average Price ($)',
    template='plotly_white'
)
fig.show()

Hotel prices are the most expensive and shared rooms are the least expensive.

In [ ]:
# price vs license

import plotly.express as px
df_license = listing.groupby('has_license', as_index=False)['price'].mean()
fig = px.bar(
    df_license,
    x='has_license',
    y='price',
    color='has_license',
    title='Average Price by License Status',
    labels={'price': 'Average Price ($)', 'has_license': 'Has License'}
)
fig.update_layout(
    xaxis_title='Has License',
    yaxis_title='Average Price ($)',
    template='plotly_white'
)
fig.show()

Houses with licences tend to avarege a higher price than houses without licences

In [ ]:
fig = px.box(
    listing,
    x='neighbourhood_group',
    y='price',
    color='neighbourhood_group',
    title='Price Distribution by Neighbourhood Group',
    labels={'price': 'Price ($)', 'neighbourhood_group': 'Neighbourhood Group'}
)
fig.update_layout(
    xaxis_title='Neighbourhood Group',
    yaxis_title='Price ($)',
    template='plotly_white'
)
fig.show()


In [ ]:
listing1 = listing.copy()

In [ ]:
import pandas as pd
import plotly.express as px

# Step 1: Create estimated 'booked_days' column
listing1['booked_days'] = 365 - listing['availability_365']

# Step 2: Average booked days per room type
df_bookings = listing1.groupby('room_type', as_index=False)['booked_days'].mean()

# Step 3: Plot
fig = px.bar(
    df_bookings,
    x='room_type',
    y='booked_days',
    color='room_type',
    title='Average Booked Days by Room Type (Estimated)',
    labels={
        'room_type': 'Room Type',
        'booked_days': 'Estimated Booked Days'
    }
)

fig.update_layout(
    xaxis_title='Room Type',
    yaxis_title='Avg. Booked Days (per Year)',
    template='plotly_white'
)

fig.show()


Majority of the people booked private rooms, and hotel rooms were the least booked, this could be as a hotel rooms being the most expensive rooms

In [ ]:
import plotly.express as px

# Group by neighbourhood and calculate average total reviews
grouped_review = listing1.groupby('neighbourhood_group', as_index=False)['total_reviews'].mean()

# Sort neighbourhoods by review count for better ordering in the plot
grouped_review = grouped_review.sort_values(by='total_reviews', ascending=True)

# Create horizontal bar chart
fig = px.bar(
    grouped_review,
    x='total_reviews',
    y='neighbourhood_group',
    orientation='h',
    title='📊 Average Total Reviews by Neighbourhood',
    color='total_reviews',
    color_continuous_scale='viridis',
    labels={'total_reviews': 'Average Total Reviews', 'neighbourhood': 'Neighbourhood'}
)

# Update layout for better spacing and readability
fig.update_layout(
    template='plotly_white',
    xaxis_title='Average Total Reviews',
    yaxis_title='Neighbourhood',
    title_x=0.5,
    margin=dict(l=100, r=30, t=80, b=40),
    height=600
)

fig.show()


Queens received the highest number of reviews, while Manhattan received the least number of reviews, Queens was the second least expensive neighboor hood, which made it prefferable for the people



In [ ]:
listing.head()

,name,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,total_reviews,reviews_per_month,host_listings_count,availability_365,number_of_reviews_ltm,has_license
1,Only 2 stops to Manhattan studio,Allen & Irina,Brooklyn,Williamsburg,40.709350,-73.953420,Entire home/apt,96.0,30,194,1.01,1,173,3,0
2,Uptown Sanctuary w/ Private Bath (Month to Month),Kahshanna,Manhattan,East Harlem,40.801070,-73.942550,Private room,59.0,30,1,0.03,2,83,0,0
3,UES Beautiful Blue Room,Cyn,Manhattan,East Harlem,40.787780,-73.947590,Private room,67.0,30,251,1.34,1,264,5,0
5,"Perfect for Your Parents, With Garden & Patio",Jane,Brooklyn,Fort Greene,40.691940,-73.973890,Private room,195.0,2,398,2.16,2,190,36,1
6,Sunny Williamsburg Loft with Sauna,Chaya,Brooklyn,Williamsburg,40.718807,-73.956177,Entire home/apt,290.0,30,13,0.07,1,0,1,0
